In [ ]:
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import random
import numpy as np
import google.datalab.storage as storage
from PIL import Image
from scipy import misc
import io
import json 
import matplotlib.pyplot as plt

In [ ]:
def generate_cut(image_matrix):
    x,y = image_matrix.shape[:2]
    x_start = random.randint(0, int(x/2))
    y_start = random.randint(0, int(y/2))
    x_end = limit(x_start + int(x/2) + random.randint(0, int(x/16)), x)
    y_end = limit(y_start + int(y/2) + random.randint(0, int(y/16)), y)
    image_cut = image_matrix[x_start:x_end, y_start:y_end]
    return image_cut
  
def limit(a, b):
  if a < b:
      return a
  else: return b

def prepare_image(test_sample):
  test_stream = io.BytesIO(test_sample)
  my_test_image = Image.open(test_stream)
  test_image_matrix = np.asarray(my_test_image)
  a1 = generate_cut(test_image_matrix)
  test_input_image = misc.imresize(a1, (40, 60)).astype("float32")
  normalized_test_image = test_input_image.ravel()/255
  test_input = np.atleast_2d(normalized_test_image)
  distortion = Image.fromarray(a1, 'RGB')
  #return test_input
  return test_input, distortion
  
                  

In [ ]:
sess = tf.Session()
new_saver = tf.train.import_meta_graph('gs://copyright/models/convolutional_network/convolutional_network.meta')
new_saver.restore(sess, tf.train.latest_checkpoint('gs://copyright/models/convolutional_network/'))
#new_saver = tf.train.import_meta_graph('convolutional_network.meta')
#new_saver.restore(sess, tf.train.latest_checkpoint('./'))

graph = tf.get_default_graph() 
x = graph.get_tensor_by_name("input:0")
y = graph.get_tensor_by_name("output:0")

In [ ]:
sample_bucket = storage.Bucket("copyright")
test_data = [_object for _object in sample_bucket.objects()
             if "train" not in _object.key and ".jpg" in _object.key and "test" not in _object.key]

test_data = test_data[0:10]
# test_sample = test_data[8].download()
# test_stream = io.BytesIO(test_sample)
# my_test_image = Image.open(test_stream)
# my_test_image

In [ ]:
with open('images.json', 'r') as f:
     data = json.load(f)

In [ ]:
def predict_source(testlist):
  sd = []
  sd_labels = []
  for i in range(len(testlist)):
    my_test_sample = testlist[i].download()
    test_input, distortion = prepare_image(my_test_sample)

    get_output = tf.argmax(y,1) 
    predicted_class = sess.run(get_output, feed_dict={x: test_input})[0]
    fname = data[str(predicted_class)][0]
    predicted_gcs_obj = [obj for obj in sample_bucket.objects(prefix = 'sources/', delimiter = '/') if fname == obj.key]
    predicted_gcs_obj_1 = predicted_gcs_obj[0].download()
    predicted_gcs_obj_stream = io.BytesIO(predicted_gcs_obj_1)
    predicted_image = Image.open(predicted_gcs_obj_stream)
    
    my_test_stream = io.BytesIO(my_test_sample)
    my_test_image = Image.open(my_test_stream)
    
    sd.append(distortion)
    sd.append(my_test_image)
    sd.append(predicted_image)
    sd_labels.append("distortion | "+ testlist[i].key)
    sd_labels.append("original | " + testlist[i].key)
    sd_labels.append("predicted | " + fname)
  return sd, sd_labels

In [ ]:
def view_results(df, variables, n_rows, n_cols):   
    fig = plt.figure(figsize=(18, 16), dpi= 80, facecolor='w', edgecolor='k')
    r = {}
    for i, var_name in enumerate(variables):
        ax = fig.add_subplot(n_rows, n_cols, i+1)
        plt.tight_layout()
        ax.imshow(df[i])
        ax.set_title("%s" % (var_name))     
        ax.grid(False)
        ax.set_xticks([])
        ax.set_yticks([])
    plt.show()
    return r

In [ ]:
sd, sd_labels = predict_source(test_data)

In [ ]:
r = view_results(sd, sd_labels, len(test_data), 3)